In [31]:
import pandas as pd
import numpy as np
from scipy.stats import shapiro, levene, f_oneway, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import scikit_posthocs as sp

In [32]:
ST_ConV_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_ConV-Capsnet.xlsx", sheet_name='Sheet1', header=0)
ST_ConV_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_ConV-FC.xlsx", sheet_name='Sheet1', header=0)
ST_Deform_Capsnet = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_Deform-Capsnet.xlsx", sheet_name='Sheet1', header=0)
ST_Deform_FC = pd.read_excel(r"D:\1_Nguyễn Duy Tân\UEH\TEAM PỂ\ST\ST_Deform-FC.xlsx", sheet_name='Sheet1', header=0)

ST_ConV_Capsnet = ST_ConV_Capsnet.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
ST_ConV_FC = ST_ConV_FC.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
ST_Deform_Capsnet = ST_Deform_Capsnet.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]
ST_Deform_FC = ST_Deform_FC.loc[:,['Dice','F1-Score','Inference_Time','Accuracy']]

## **1.Kiểm định cho SEGMENTATION**

In [33]:
ST_Dice = pd.DataFrame(
    {
        'ConV_Capsnet': ST_ConV_Capsnet['Dice'],
        'ConV_FC': ST_ConV_FC['Dice'],
        'Deform_Capsnet': ST_Deform_Capsnet['Dice'],
        'Deform_FC': ST_Deform_FC['Dice']  
    }
)

ST_Dice.head()

,ConV_Capsnet,ConV_FC,Deform_Capsnet,Deform_FC
0,0.7658,0.755972,0.591796,0.7067
1,0.7705,0.737258,0.628160,0.6580
2,0.7677,0.759692,0.636533,0.6474
3,0.7643,0.750755,0.649629,0.6755
4,0.7573,0.764791,0.585342,0.6463


### **1.1.Kiểm định Phân phối chuẩn (Shapiro-Wilk)**

In [34]:
alpha = 0.05
is_normal_seg = True

for col in ST_Dice.columns:
    stat_seg, p_shapiro_seg = shapiro(ST_Dice[col])
    print(f'{col}: Statistic={stat_seg:.4f}, p-value={p_shapiro_seg:.4f}')
    
    if p_shapiro_seg > alpha:
        print(f'{col} CÓ THỂ COI là phân phối chuẩn (fail to reject H0)\n')
    else:
        print(f'{col} KHÔNG PHẢI là phân phối chuẩn (reject H0)\n')
        is_normal_seg = False


ConV_Capsnet: Statistic=0.9079, p-value=0.0132
ConV_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

ConV_FC: Statistic=0.8797, p-value=0.0028
ConV_FC KHÔNG PHẢI là phân phối chuẩn (reject H0)

Deform_Capsnet: Statistic=0.9368, p-value=0.0746
Deform_Capsnet CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_FC: Statistic=0.9377, p-value=0.0790
Deform_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)



### **1.2.Kiểm định Kruskal-Wallis**

In [35]:
# Nếu dữ liệu phân phối không chuẩn, thực hiện kiểm định Kruskal-Wallis
if is_normal_seg == False:
    seg_announce = ' SEQUENTIAL LEARNING (ST) - ĐÁNH GIÁ DICE SCORE CHO SEGMENTATION: '
    Seg_Announce = seg_announce.center(80, '-')
    print(Seg_Announce)
    print('\nVì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.\n')
    
    # Kiểm định Kruskal-Wallis
    h_stat, p_kruskal = kruskal(
    ST_Dice['ConV_Capsnet'],
    ST_Dice['ConV_FC'],
    ST_Dice['Deform_Capsnet'],
    ST_Dice['Deform_FC']
    )

    # In kết quả kiểm định Kruskal-Wallis
    print("-- Kiểm định Kruskal-Wallis cho ST-Dice: ---")
    print(f"Thống kê H (H-statistic): {h_stat:.4f}")
    print(f"Giá trị p (p-value): {p_kruskal:.4f}")

    if p_kruskal < alpha:
        print("\nCó bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).")
        print("Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.")

        # Chuyển dữ liệu sang dạng "long" để dùng cho posthoc_dunn
        dice_long = pd.melt(ST_Dice.reset_index(), id_vars=['index'], value_vars=ST_Dice.columns,
                            var_name='Model', value_name='Dice')
        dunn_result = sp.posthoc_dunn(dice_long, group_col='Model', val_col='Dice', p_adjust='bonferroni')

        # In kết quả kiểm định Dunn's Post-Hoc
        print("\n--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---")
        print(dunn_result.round(4))

        print("\n--- Kết quả trung bình Dice Score cho từng mô hình ---")
        print(f'ConV_Capsnet:\t {ST_ConV_Capsnet["Dice"].mean():.4f}')
        print(f'ConV_FC:\t {ST_ConV_FC["Dice"].mean():.4f}')
        print(f'Deform_Capsnet:\t {ST_Deform_Capsnet["Dice"].mean():.4f}') 
        print(f'Deform_FC:\t {ST_Deform_FC["Dice"].mean():.4f}')
    
    else:
        print("\nKhông có đủ bằng chứng để kết luận có sự khác biệt về hiệu suất giữa các mô hình (p >= 0.05).")
        print("Dừng phân tích tại đây.")

significant_pairs = []
non_significant_pairs = []

for i in range(len(dunn_result.index)):
    for j in range(i+1, len(dunn_result.columns)):
        p = dunn_result.iloc[i, j]
        pair = (dunn_result.index[i], dunn_result.columns[j])
        mean1 = ST_Dice[pair[0]].mean()  # Nếu dùng Dice thì thay ST_F1 bằng ST_Dice
        mean2 = ST_Dice[pair[1]].mean()
        entry = {
            'Model 1': pair[0],
            'Model 2': pair[1],
            'p-value': round(p, 4),
            'Mean Model 1': round(mean1, 4),
            'Mean Model 2': round(mean2, 4),
            'Ý nghĩa': 'Có' if p < 0.05 else 'Không'
        }
        if p < 0.05:
            significant_pairs.append(entry)
        else:
            non_significant_pairs.append(entry)

seg_pairs = pd.DataFrame(significant_pairs + non_significant_pairs)
        

------- SEQUENTIAL LEARNING (ST) - ĐÁNH GIÁ DICE SCORE CHO SEGMENTATION: -------

Vì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.

-- Kiểm định Kruskal-Wallis cho ST-Dice: ---
Thống kê H (H-statistic): 91.0495
Giá trị p (p-value): 0.0000

Có bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).
Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.

--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---
                ConV_Capsnet  ConV_FC  Deform_Capsnet  Deform_FC
ConV_Capsnet             1.0      1.0          0.0000     0.0000
ConV_FC                  1.0      1.0          0.0000     0.0000
Deform_Capsnet           0.0      0.0          1.0000     0.3685
Deform_FC                0.0      0.0          0.3685     1.0000

--- Kết quả trung bình Dice Score cho từng mô hình ---
ConV_Capsnet:	 0.7611
ConV_FC:	 0.7626
Deform_Capsnet:	 0.6366
Deform_FC:	 0.6607


## **2.Kiểm định cho CLASSIFICATION**

In [36]:
ST_F1 = pd.DataFrame(
    {
    'ConV_Capsnet': ST_ConV_Capsnet['F1-Score'],
    'ConV_FC': ST_ConV_FC['F1-Score'],
    'Deform_Capsnet': ST_Deform_Capsnet['F1-Score'],
    'Deform_FC': ST_Deform_FC['F1-Score']
    })

ST_F1.head()

,ConV_Capsnet,ConV_FC,Deform_Capsnet,Deform_FC
0,0.7155,0.796378,0.603351,0.5993
1,0.4274,0.732760,0.536457,0.6237
2,0.7296,0.758514,0.433222,0.5572
3,0.4531,0.743206,0.666065,0.6698
4,0.4321,0.743076,0.405305,0.5802


In [37]:
is_normal_cls = True

for col in ST_F1.columns:
    stat_cls, p_shapiro_cls = shapiro(ST_F1[col])
    print(f'{col}: Statistic={stat_cls:.4f}, p-value={p_shapiro_cls:.4f}')
    
    if p_shapiro_cls > alpha:
        print(f'{col} CÓ THỂ COI là phân phối chuẩn (fail to reject H0)\n')
    else:
        print(f'{col} KHÔNG PHẢI là phân phối chuẩn (reject H0)\n')
        is_normal_cls = False

ConV_Capsnet: Statistic=0.7590, p-value=0.0000
ConV_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

ConV_FC: Statistic=0.9794, p-value=0.8098
ConV_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_Capsnet: Statistic=0.9376, p-value=0.0784
Deform_Capsnet CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_FC: Statistic=0.8809, p-value=0.0030
Deform_FC KHÔNG PHẢI là phân phối chuẩn (reject H0)



In [38]:
# Nếu dữ liệu phân phối không chuẩn, thực hiện kiểm định Kruskal-Wallis
if is_normal_cls == False:
    cls_announce = ' SEQUENTIAL LEARNING (ST) - ĐÁNH GIÁ F1 SCORE CHO CLASSIFICATION: '
    Cls_Announce = cls_announce.center(80, '-')
    print(Cls_Announce)
    print('\nVì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.\n')
    
    # Kiểm định Kruskal-Wallis
    h_stat, p_kruskal = kruskal(
    ST_F1['ConV_Capsnet'],
    ST_F1['ConV_FC'],
    ST_F1['Deform_Capsnet'],
    ST_F1['Deform_FC']
    )

    # In kết quả kiểm định Kruskal-Wallis
    print("-- Kiểm định Kruskal-Wallis cho ST-F1_Score: ---")
    print(f"Thống kê H (H-statistic): {h_stat:.4f}")
    print(f"Giá trị p (p-value): {p_kruskal:.4f}")

    if p_kruskal < alpha:
        print("\nCó bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).")
        print("Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.")

        # Chuyển dữ liệu sang dạng "long" để dùng cho posthoc_dunn
        f1_long = pd.melt(ST_F1.reset_index(), id_vars=['index'], value_vars=ST_F1.columns,
                            var_name='Model', value_name='F1-Score')
        dunn_result = sp.posthoc_dunn(f1_long, group_col='Model', val_col='F1-Score', p_adjust='bonferroni')

        # In kết quả kiểm định Dunn's Post-Hoc
        print("\n--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---")
        print(dunn_result.round(4))

        print("\n--- Kết quả trung bình F1-Score cho từng mô hình ---")
        print(f'ConV_Capsnet:\t {ST_ConV_Capsnet["F1-Score"].mean():.4f}')
        print(f'ConV_FC:\t {ST_ConV_FC["F1-Score"].mean():.4f}')
        print(f'Deform_Capsnet:\t {ST_Deform_Capsnet["F1-Score"].mean():.4f}') 
        print(f'Deform_FC:\t {ST_Deform_FC["F1-Score"].mean():.4f}')
    
    else:
        print("\nKhông có đủ bằng chứng để kết luận có sự khác biệt về hiệu suất giữa các mô hình (p >= 0.05).")
        print("Dừng phân tích tại đây.")

significant_pairs = []
non_significant_pairs = []

for i in range(len(dunn_result.index)):
    for j in range(i+1, len(dunn_result.columns)):
        p = dunn_result.iloc[i, j]
        pair = (dunn_result.index[i], dunn_result.columns[j])
        mean1 = ST_F1[pair[0]].mean()  # Nếu dùng Dice thì thay ST_F1 bằng ST_Dice
        mean2 = ST_F1[pair[1]].mean()
        entry = {
            'Model 1': pair[0],
            'Model 2': pair[1],
            'p-value': round(p, 4),
            'Mean Model 1': round(mean1, 4),
            'Mean Model 2': round(mean2, 4),
            'Ý nghĩa': 'Có' if p < 0.05 else 'Không'
        }
        if p < 0.05:
            significant_pairs.append(entry)
        else:
            non_significant_pairs.append(entry)

cls_pairs = pd.DataFrame(significant_pairs + non_significant_pairs)
    
        

------- SEQUENTIAL LEARNING (ST) - ĐÁNH GIÁ F1 SCORE CHO CLASSIFICATION: -------

Vì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.

-- Kiểm định Kruskal-Wallis cho ST-F1_Score: ---
Thống kê H (H-statistic): 52.8637
Giá trị p (p-value): 0.0000

Có bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).
Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.

--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---
                ConV_Capsnet  ConV_FC  Deform_Capsnet  Deform_FC
ConV_Capsnet          1.0000   0.0019          0.0288     0.0742
ConV_FC               0.0019   1.0000          0.0000     0.0000
Deform_Capsnet        0.0288   0.0000          1.0000     1.0000
Deform_FC             0.0742   0.0000          1.0000     1.0000

--- Kết quả trung bình F1-Score cho từng mô hình ---
ConV_Capsnet:	 0.6169
ConV_FC:	 0.7256
Deform_Capsnet:	 0.5574
Deform_FC:	 0.5763


In [39]:
# Kết quả cho Segmentation
seg_pairs

,Model 1,Model 2,p-value,Mean Model 1,Mean Model 2,Ý nghĩa
0,ConV_Capsnet,Deform_Capsnet,0.0000,0.7611,0.6366,Có
1,ConV_Capsnet,Deform_FC,0.0000,0.7611,0.6607,Có
2,ConV_FC,Deform_Capsnet,0.0000,0.7626,0.6366,Có
3,ConV_FC,Deform_FC,0.0000,0.7626,0.6607,Có
4,ConV_Capsnet,ConV_FC,1.0000,0.7611,0.7626,Không
5,Deform_Capsnet,Deform_FC,0.3685,0.6366,0.6607,Không


In [40]:
# Kết quả cho Classification
cls_pairs

,Model 1,Model 2,p-value,Mean Model 1,Mean Model 2,Ý nghĩa
0,ConV_Capsnet,ConV_FC,0.0019,0.6169,0.7256,Có
1,ConV_Capsnet,Deform_Capsnet,0.0288,0.6169,0.5574,Có
2,ConV_FC,Deform_Capsnet,0.0000,0.7256,0.5574,Có
3,ConV_FC,Deform_FC,0.0000,0.7256,0.5763,Có
4,ConV_Capsnet,Deform_FC,0.0742,0.6169,0.5763,Không
5,Deform_Capsnet,Deform_FC,1.0000,0.5574,0.5763,Không


### **Accuracy**

In [41]:
ST_Acc = pd.DataFrame(
    {
    'ConV_Capsnet': ST_ConV_Capsnet['Accuracy'],
    'ConV_FC': ST_ConV_FC['Accuracy'],
    'Deform_Capsnet': ST_Deform_Capsnet['Accuracy'],
    'Deform_FC': ST_Deform_FC['Accuracy']
    })

ST_Acc.head()

,ConV_Capsnet,ConV_FC,Deform_Capsnet,Deform_FC
0,0.7098,0.803746,0.579487,0.5891
1,0.4535,0.731193,0.524205,0.6256
2,0.7347,0.754668,0.450769,0.5533
3,0.4901,0.734446,0.639208,0.6609
4,0.4573,0.755823,0.422034,0.5826


In [42]:

for col in ST_Acc.columns:
    stat_cls, p_shapiro_cls = shapiro(ST_Acc[col])
    print(f'{col}: Statistic={stat_cls:.4f}, p-value={p_shapiro_cls:.4f}')
    
    if p_shapiro_cls > alpha:
        print(f'{col} CÓ THỂ COI là phân phối chuẩn (fail to reject H0)\n')
    else:
        print(f'{col} KHÔNG PHẢI là phân phối chuẩn (reject H0)\n')



ConV_Capsnet: Statistic=0.7728, p-value=0.0000
ConV_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

ConV_FC: Statistic=0.9844, p-value=0.9273
ConV_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)

Deform_Capsnet: Statistic=0.9188, p-value=0.0250
Deform_Capsnet KHÔNG PHẢI là phân phối chuẩn (reject H0)

Deform_FC: Statistic=0.9406, p-value=0.0944
Deform_FC CÓ THỂ COI là phân phối chuẩn (fail to reject H0)



In [43]:
# Nếu dữ liệu phân phối không chuẩn, thực hiện kiểm định Kruskal-Wallis
if True:
    acc_announce = ' SEQUENTIAL LEARNING (ST) - ĐÁNH GIÁ ACCURACY CHO CLASSIFICATION: '
    acc_Announce = acc_announce.center(80, '-')
    print(acc_Announce)
    print('\nVì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.\n')
    
    # Kiểm định Kruskal-Wallis
    h_stat, p_kruskal = kruskal(
    ST_Acc['ConV_Capsnet'],
    ST_Acc['ConV_FC'],
    ST_Acc['Deform_Capsnet'],
    ST_Acc['Deform_FC']
    )

    # In kết quả kiểm định Kruskal-Wallis
    print("-- Kiểm định Kruskal-Wallis cho ST-Accuracy: ---")
    print(f"Thống kê H (H-statistic): {h_stat:.4f}")
    print(f"Giá trị p (p-value): {p_kruskal:.4f}")

    if p_kruskal < alpha:
        print("\nCó bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).")
        print("Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.")

        # Chuyển dữ liệu sang dạng "long" để dùng cho posthoc_dunn
        Acc_long = pd.melt(ST_Acc.reset_index(), id_vars=['index'], value_vars=ST_Acc.columns,
                            var_name='Model', value_name='Accuracy')
        dunn_result = sp.posthoc_dunn(Acc_long, group_col='Model', val_col='Accuracy', p_adjust='bonferroni')

        # In kết quả kiểm định Dunn's Post-Hoc
        print("\n--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---")
        print(dunn_result.round(4))

        print("\n--- Kết quả trung bình Accuracy cho từng mô hình ---")
        print(f'ConV_Capsnet:\t {ST_ConV_Capsnet["Accuracy"].mean():.4f}')
        print(f'ConV_FC:\t {ST_ConV_FC["Accuracy"].mean():.4f}')
        print(f'Deform_Capsnet:\t {ST_Deform_Capsnet["Accuracy"].mean():.4f}') 
        print(f'Deform_FC:\t {ST_Deform_FC["Accuracy"].mean():.4f}')
    
    else:
        print("\nKhông có đủ bằng chứng để kết luận có sự khác biệt về hiệu suất giữa các mô hình (p >= 0.05).")
        print("Dừng phân tích tại đây.")

significant_pairs = []
non_significant_pairs = []

for i in range(len(dunn_result.index)):
    for j in range(i+1, len(dunn_result.columns)):
        p = dunn_result.iloc[i, j]
        pair = (dunn_result.index[i], dunn_result.columns[j])
        mean1 = ST_Acc[pair[0]].mean()  # Nếu dùng Dice thì thay ST_Acc bằng ST_Dice
        mean2 = ST_Acc[pair[1]].mean()
        entry = {
            'Model 1': pair[0],
            'Model 2': pair[1],
            'p-value': round(p, 4),
            'Mean Model 1': round(mean1, 4),
            'Mean Model 2': round(mean2, 4),
            'Ý nghĩa': 'Có' if p < 0.05 else 'Không'
        }
        if p < 0.05:
            significant_pairs.append(entry)
        else:
            non_significant_pairs.append(entry)

Acc_pairs = pd.DataFrame(significant_pairs + non_significant_pairs)
    
        

------- SEQUENTIAL LEARNING (ST) - ĐÁNH GIÁ ACCURACY CHO CLASSIFICATION: -------

Vì dữ liệu vi phạm giả định phân phối chuẩn, nên chuyển sang Kiểm định phi tham số Kruskal-Wallis.

-- Kiểm định Kruskal-Wallis cho ST-Accuracy: ---
Thống kê H (H-statistic): 55.3091
Giá trị p (p-value): 0.0000

Có bằng chứng cho thấy ít nhất một mô hình có hiệu suất khác biệt (p-value < 0.05).
Tiến hành kiểm định Dunn's Post-Hoc để so sánh từng cặp mô hình.

--- Bảng kết quả kiểm định Dunn's Post-Hoc (đã hiệu chỉnh Bonferroni) ---
                ConV_Capsnet  ConV_FC  Deform_Capsnet  Deform_FC
ConV_Capsnet          1.0000   0.0029          0.0058     0.0941
ConV_FC               0.0029   1.0000          0.0000     0.0000
Deform_Capsnet        0.0058   0.0000          1.0000     1.0000
Deform_FC             0.0941   0.0000          1.0000     1.0000

--- Kết quả trung bình Accuracy cho từng mô hình ---
ConV_Capsnet:	 0.6241
ConV_FC:	 0.7183
Deform_Capsnet:	 0.5530
Deform_FC:	 0.5832


In [44]:
Acc_pairs

,Model 1,Model 2,p-value,Mean Model 1,Mean Model 2,Ý nghĩa
0,ConV_Capsnet,ConV_FC,0.0029,0.6241,0.7183,Có
1,ConV_Capsnet,Deform_Capsnet,0.0058,0.6241,0.5530,Có
2,ConV_FC,Deform_Capsnet,0.0000,0.7183,0.5530,Có
3,ConV_FC,Deform_FC,0.0000,0.7183,0.5832,Có
4,ConV_Capsnet,Deform_FC,0.0941,0.6241,0.5832,Không
5,Deform_Capsnet,Deform_FC,1.0000,0.5530,0.5832,Không
